# Projeto 1 - Ciência dos Dados

Nome: Layne Silva

Nome: Lidia Domingos

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [525]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import nltk 
from nltk.stem import RSLPStemmer
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lidia\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [526]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\lidia\Desktop\Ciência dos Dados\Projeto1\PROJETO-1-CDADOS


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [527]:
filename = 'XIAOMI.xlsx'

In [528]:
train = pd.read_excel(filename)
train.head(5)

,TREINAMENTO,CLASSIFICAÇÃO
0,quem aqui tb pronuncia xiaomi xaiomi,0
1,xiaomi mi true airdots 2s earphone(aliexpress)...,0
2,@felpsflip carreguei meu xiaomi uma vez na vid...,1
3,"#promo #\n$133.99 ~ 111,85 € ✂️cupão/cupom/cup...",0
4,"#promo #\n$299.99 ~ 250,41 € ✂️cupão/cupom/cup...",0


In [529]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,CLASSIFICAÇÃO
0,"#promo #\n$11.99 ~ 10,01 € ✂️cupão/cupom/cupon...",0
1,"💠xiaomi poco m3 \n\n💵 por: r$ 728,27 64gb\n💵 ...",0
2,"@_faccin_ ""quem ai tem um xiaomi pra ligar o p...",0
3,comprei mais capinha pro xiaomi https://t.co/7...,0
4,#promoção #oferta \n\n💵 por: r$ 1420 \n🛒 link:...,0


In [530]:
test.iloc[:,0:1]

,Teste
0,"#promo #\n$11.99 ~ 10,01 € ✂️cupão/cupom/cupon..."
1,"💠xiaomi poco m3 \n\n💵 por: r$ 728,27 64gb\n💵 ..."
2,"@_faccin_ ""quem ai tem um xiaomi pra ligar o p..."
3,comprei mais capinha pro xiaomi https://t.co/7...
4,#promoção #oferta \n\n💵 por: r$ 1420 \n🛒 link:...
...,...
254,"rt @victorxama: salve rapa, tô vendendo meu xi..."
255,@mslaryhill tenho um haylou gt1 pro q é da mes...
256,@gabiiii__araujo 😱😱😱\n\n*baixou mais!!*\n\n*xi...
257,@inutializador @viniport0 @2kjuramento @pessoa...


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

ESCREVA AQUI...

*Escolhida a empresa XIAOMI, classificamos como tweets relevantes toda e qualquer citação (seja ela positiva ou negativa) dos internautas falando sobre o produto. Tweets considerados como irrelevantes seriam tweets não relacionados diretamente ao produto.*

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [550]:
#Limpeza de mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.
#Correção de espaços entre palavras e/ou emojis.
#Proposta de outras limpezas/transformações que não afetem a qualidade da informação.

import re 
from nltk.tokenize.casual import TweetTokenizer #separa os emojis como palavras.

def cleanup(text):
    #import string
    punctuation = '[!-@.~[\]:"''?;\),("/]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    text_subbed = re.sub(r'http\S+', '',text_subbed)
    t = TweetTokenizer()
    text_subbed = t.tokenize(text_subbed) 
    return text_subbed

def RemoveStopWords(list_of_words):
    stopwords = nltk.corpus.stopwords.words('portuguese')
    lista_filtrada = []
    for word in list_of_words:
        if word not in stopwords:
            lista_filtrada.append(word)
    return lista_filtrada

def Suavizacao_Laplace(palavra1, palavra2, palavra3):
    y = (1 + palavra1)/(len(palavra2) + palavra3)  
    return y

def MultiplyList(List) : 
    result = 1
    for x in List: 
         result = result * x  
    return result



In [551]:
#JUNTANDO TODAS AS PALAVRAS DOS TWEETS RELEVANTES DO TREINAMENTO
train_relevante = train.loc[train["CLASSIFICAÇÃO"] == 1, :]
train_relevante_filtrada = ' '.join(train_relevante.TREINAMENTO) 

#JUNTANDO TODAS AS PALAVRAS DOS TWEETS IRRELEVANTES DO TREINAMENTO
train_irrelevante = train.loc[train["CLASSIFICAÇÃO"] == 0, :]
train_irrelevante_filtrada = ' '.join(train_irrelevante.TREINAMENTO) 


In [552]:
#LIMPANDO VARIÁVEIS CRIADAS, COLOCANDO EM MINÚSCULO E AS GUARDANDO
relevante = cleanup(train_relevante_filtrada.lower())
palavras_relevantes = RemoveStopWords(relevante)

irrelevante = cleanup(train_irrelevante_filtrada.lower())
palavras_irrelevantes = RemoveStopWords(irrelevante)

#JUNTANDO OS CONJUNTOS
palavras_total = palavras_relevantes + palavras_irrelevantes


In [553]:
#TRANSFORMANDO EM PD.SERIES
series_total = pd.Series(palavras_total)
series_train_relevante = pd.Series(palavras_relevantes)
series_train_irrelevante = pd.Series(palavras_irrelevantes)


In [554]:
#FREQUÊNCIAS RELATIVAS
palavras_total_relativas = series_total.value_counts(True)
palavras_relevante_relativas = series_train_relevante.value_counts(True)
palavras_irrelevante_relativas = series_train_irrelevante.value_counts(True)


In [555]:
#FREQUÊNCIAS ABSOLUTAS
palavras_relevante_absolutas = series_train_relevante.value_counts()
palavras_irrelevante_absolutas = series_train_irrelevante.value_counts()
palavras_total_absolutas = series_total.value_counts()


**EVENTOS**

 * $R$: comentários relevantes
 * $R^c$: comentários irrelevantes
 * $C$: comentário recebido

**PROBABILIDADES**

 * $P(R)$ : probabilidade do comentário ser relevante ;
 * $P(R^c)$: probabilidade do comentário não ser relevante, ou seja, irrelevante ;
 * $P(C)$ : probabilidade de cada palavra ocorrer na língua portuguesa ;
 * $P(C|R)$ : probabilidade do comentário existir dado o conjunto de relevantes ;
 * $P(C|R^c)$: probabilidade do comentário existir dado o conjunto de irrelevantes ; 
 * $P(R|C)$: probabilidade do comentário ser relevante dado a frase ;
 * $P(R^c|C)$: probabilidade do comentário ser irrelevante dado a frase;


**CLASSIFICADOR**



In [578]:
def funcao_classificadora(test):
    #separando os comentários
    
    test['classificação_nova'] = 'NaN'
    
#     comentarios = test.iloc[:,0:1]
#     comentarios_juntos = ' '.join(comentarios.iloc[:,0]) 
#     palavras_limpas_comentario = RemoveStopWords(cleanup(comentarios_juntos.lower()))
#     palavras_serie_comentario = pd.Series(palavras_limpas_comentario)
#     palavras_comentarios = palavras_serie_comentario.value_counts()
    
    for tweet in list(test.Teste):
        tweet_limpo = RemoveStopWords(cleanup(tweet.lower()))
        lista_de_probabilidades1 = []
        lista_de_probabilidades2 = []
        for palavra in tweet_limpo:
            if palavra in palavras_relevante_absolutas:
                Laplace = Suavizacao_Laplace(palavras_relevante_absolutas[palavra],palavras_total_absolutas,palavras_relevante_absolutas.sum())
                lista_de_probabilidades1.append(Laplace)
            if palavra == 'xiaomi':
                continue
            else:
                Laplace = Suavizacao_Laplace(0,palavras_total_absolutas,palavras_relevante_absolutas.sum())
                lista_de_probabilidades1.append(Laplace)
            P_C_dado_R = MultiplyList(lista_de_probabilidades1)

        for palavra in tweet_limpo:
            if palavra in palavras_irrelevante_absolutas:
                Laplace = Suavizacao_Laplace(palavras_irrelevante_absolutas[palavra],palavras_total_absolutas,palavras_irrelevante_absolutas.sum())
                lista_de_probabilidades2.append(Laplace)
            if palavra == 'xiaomi':
                continue
            else:
                Laplace = Suavizacao_Laplace(0,palavras_total_absolutas,palavras_irrelevante_absolutas.sum())
                lista_de_probabilidades2.append(Laplace)
            P_C_dado_Rc = MultiplyList(lista_de_probabilidades2)
        
        P_R = (palavras_relevante_absolutas.sum())/(palavras_total_absolutas.sum())
    
        P_Rc = 1 - P_R
        
        P_R_dado_C = P_R * P_C_dado_R

        P_Rc_dado_C = P_Rc * P_C_dado_Rc            
        
        if P_R_dado_C > P_Rc_dado_C:
            resultado = 1
        else:
            resultado = 0
            
        test['classificação_nova'] = resultado

    return test
    

In [579]:
teste = funcao_classificadora(test)
teste.loc[teste.classificação_nova == 1,:]

,Teste,CLASSIFICAÇÃO,classificação_nova


In [569]:
test['classificação_nova'] = 'NaN'
lista = []
lista1 = []
comentarios = test.iloc[:,0:1]
comentarios_juntos = ' '.join(comentarios.iloc[:,0]) 
palavras_limpas_comentario = RemoveStopWords(cleanup(comentarios_juntos.lower()))
palavras_serie_comentario = pd.Series(palavras_limpas_comentario)
palavras_comentarios = palavras_serie_comentario.value_counts()


for tweet in list(test.Teste):
    tweet_limpo = RemoveStopWords(cleanup(tweet.lower()))
    lista_de_probabilidades1 = []
    lista_de_probabilidades2 = []
    for palavra in tweet_limpo:
        if palavra in palavras_relevante_absolutas:
            Laplace = Suavizacao_Laplace(palavras_relevante_absolutas[palavra],palavras_total_absolutas,palavras_relevante_absolutas.sum())
            lista_de_probabilidades1.append(Laplace)
        if palavra == 'xiaomi':
            continue
        else:
            Laplace = Suavizacao_Laplace(0,palavras_total_absolutas,palavras_relevante_absolutas.sum())
            lista_de_probabilidades1.append(Laplace)
        P_C_dado_R = MultiplyList(lista_de_probabilidades1)

    for palavra in tweet_limpo:
        if palavra in palavras_irrelevante_absolutas:
            Laplace = Suavizacao_Laplace(palavras_irrelevante_absolutas[palavra],palavras_total_absolutas,palavras_irrelevante_absolutas.sum())
            lista_de_probabilidades2.append(Laplace)
        if palavra == 'xiaomi':
            continue
        else:
            Laplace = Suavizacao_Laplace(0,palavras_total_absolutas,palavras_irrelevante_absolutas.sum())
            lista_de_probabilidades2.append(Laplace)
        P_C_dado_Rc = MultiplyList(lista_de_probabilidades2)
        
    P_R = (palavras_relevante_absolutas.sum())/(palavras_total_absolutas.sum())
    
    P_Rc = 1 - P_R

    P_R_dado_C = P_R * P_C_dado_R

    P_Rc_dado_C = P_Rc * P_C_dado_Rc            

    if P_R_dado_C > P_Rc_dado_C:
        resultado = 1
    else:
        resultado = 0
    lista.append(P_R_dado_C)
    lista1.append(P_Rc_dado_C)
    test['classificação_nova'] = resultado


In [570]:
lista

[1.6271096070509917e-41,
 4.1417670222447904e-66,
 2.5831992121541533e-37,
 3.8843724928550114e-17,
 1.4829506617675967e-44,
 1.327418136646568e-25,
 4.510776930126956e-134,
 4.59004730684893e-90,
 2.0994920681700356e-34,
 4.688028199835317e-37,
 2.860810331756388e-46,
 6.516468978410222e-50,
 2.224425992651396e-45,
 8.692255149029328e-122,
 1.0619345093172549e-24,
 5.1437555670165864e-33,
 6.31205294190106e-62,
 3.9409121917052286e-48,
 1.0081760911352512e-30,
 2.528731550311713e-22,
 8.007933573545024e-44,
 2.302087963756831e-138,
 6.378269659639886e-39,
 9.851049098125806e-53,
 4.2145525838528543e-22,
 7.477193775930345e-116,
 8.096333444025196e-90,
 1.1402395412791284e-53,
 4.208035294600707e-62,
 1.1379291976402707e-21,
 6.467050764979347e-128,
 6.906414292296209e-13,
 3.440451088859473e-24,
 2.799672672904742e-30,
 1.6834245196050128e-57,
 2.6838650304146237e-62,
 6.414916026880188e-59,
 3.5084584604864747e-09,
 1.5612405805608609e-93,
 3.422933462671746e-127,
 3.4000350421103174

In [566]:
lista1

[2.6603148740908835e-66,
 1.0430502559809033e-98,
 1.0778619119182692e-37,
 6.75053196406449e-20,
 9.801337790142569e-59,
 1.6101800447804582e-23,
 1.5068612741319963e-195,
 3.576698280217138e-90,
 2.8086849724024806e-38,
 1.650998914350197e-44,
 5.155159875420003e-45,
 8.482018215101941e-48,
 1.9467560894832257e-45,
 3.274099047778976e-200,
 4.617075351840688e-25,
 2.4638034238800995e-34,
 7.813470876247962e-84,
 2.99966410875366e-53,
 1.5078476954146204e-31,
 3.197974269673204e-26,
 2.1901006006686296e-45,
 5.1233705798134594e-216,
 3.82293232354782e-36,
 5.234107781600773e-54,
 1.4491620403024125e-22,
 6.728906807355294e-163,
 2.5390307683961565e-106,
 9.574022813175411e-59,
 5.411964742341738e-94,
 7.195442106764708e-26,
 5.228798541338039e-179,
 1.1756170542452969e-15,
 1.3328991544731256e-18,
 1.5728635845453885e-37,
 1.7560908154647943e-58,
 1.1297695909399013e-66,
 3.3558807299461107e-69,
 8.536734089444066e-06,
 6.61427936596577e-146,
 4.1733255986110643e-128,
 9.1551303921842

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [430]:
funcao_classificadora(test)

,Teste,CLASSIFICAÇÃO,classificação_nova
0,"#promo #\n$11.99 ~ 10,01 € ✂️cupão/cupom/cupon...",0,0
1,"💠xiaomi poco m3 \n\n💵 por: r$ 728,27 64gb\n💵 ...",0,0
2,"@_faccin_ ""quem ai tem um xiaomi pra ligar o p...",0,0
3,comprei mais capinha pro xiaomi https://t.co/7...,0,0
4,#promoção #oferta \n\n💵 por: r$ 1420 \n🛒 link:...,0,0
...,...,...,...
254,"rt @victorxama: salve rapa, tô vendendo meu xi...",0,0
255,@mslaryhill tenho um haylou gt1 pro q é da mes...,1,0
256,@gabiiii__araujo 😱😱😱\n\n*baixou mais!!*\n\n*xi...,0,0
257,@inutializador @viniport0 @2kjuramento @pessoa...,0,0


In [568]:
P_R = (palavras_relevante_absolutas.sum())/(palavras_total_absolutas.sum())

0.29333333333333333

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)


___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**